In [1]:
import flopy
import flopy.mf6 as mf6
import flopy.plot as fplt

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles

Fun fact: Flopy and modflow do not support multi-solute transport in the same gwt model. Thus, we have to create separate GWT model objects for each solute. 

First, let's load in the simulation and the models created up to this point:

# TODO: ADD OBS WELLS TO EACH GWT MODEL

In [2]:
sim = mf6.MFSimulation.load(
    sim_name='peterson',
    exe_name='../../mf6/bin/mf6.exe',
    sim_ws='../input-files/'
)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package riv...
    loading package obs...
    loading package chd...
    loading package sto...
    loading package rch...
    loading package oc...
    loading package rch...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package adv...
    loading package dsp...
    loading package mst...
    loading package ist...
    loading package ssm...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package peterson_flow...
  loading solution package peterson_pfos...


In [3]:
gwf = sim.get_model('peterson_flow')
gwt_pfos = sim.get_model('peterson_pfos')

## Create New Transport Models

In [4]:
pfoa_name = 'peterson_pfoa'
pfoa_nam_file = f"{pfoa_name}.nam"

gwt_pfoa = mf6.ModflowGwt(
    sim,
    modelname=pfoa_name,
    model_nam_file=pfoa_nam_file
)

pfhxs_name = 'peterson_pfhxs'
pfhxs_nam_file = f"{pfhxs_name}.nam"

gwt_pfhxs = mf6.ModflowGwt(
    sim,
    modelname=pfhxs_name,
    model_nam_file=pfhxs_nam_file
)

## Register redundant packages to new models

Most of the models have redundant packages and only need a few to be changed. The transport model for PFOS has the following parameters:

In [5]:
gwt_pfos.package_names

['dis', 'ic', 'adv', 'dsp', 'mst', 'ist', 'ssm', 'oc']

Redundant packages will be loaded and assigned to the new model objects. The unique packages will be created for each solute.

Redundant packages:
- `dis`
- `ic`
- `adv`,
- `dsp`,
- `ist`,

Model unique packages:
- `mst`
- `ssm`
- `oc`

There is also the `ims` package. Modflow requires a new IMS for each model, but we are using the same parameters for each model. We will load this in and assign it to each of the new solute models just like we did in the previous notebook.

### Register IMS Package for each model

In [6]:
# IMS package for the GWT model
ims_pfoa = flopy.mf6.ModflowIms(
    sim,
    pname='ims_pfoa',
    filename='peterson_pfoa.ims',
    complexity='COMPLEX',
    linear_acceleration='BICGSTAB',
    outer_maximum=100,
    inner_maximum=500,
    inner_dvclose=0.1,
    outer_dvclose=0.1,
    rcloserecord=8460, 
    no_ptcrecord='FIRST'
)
sim.register_ims_package(ims_pfoa, [pfoa_name])  # Assign ims_gwt to the GWT model

In [7]:
# IMS package for the GWT model
ims_pfhxs = flopy.mf6.ModflowIms(
    sim,
    pname='ims_pfhxs',
    filename='peterson_pfhxs.ims',
    complexity='COMPLEX',
    linear_acceleration='BICGSTAB',
    outer_maximum=100,
    inner_maximum=500,
    inner_dvclose=0.1,
    outer_dvclose=0.1,
    rcloserecord=8460, 
    no_ptcrecord='FIRST'
)
sim.register_ims_package(ims_pfhxs, [pfhxs_name])  # Assign ims_gwt to the GWT model

### Write Redundant Package Information

In [8]:
# Get redundant packages
dis = gwt_pfos.get_package('dis')
ic  = gwt_pfos.get_package('ic')
adv = gwt_pfos.get_package('adv')
dsp = gwt_pfos.get_package('dsp')
ist = gwt_pfos.get_package('ist')

# Place in list for iteration
redundant_packages = [dis, ic, adv, dsp, ist]

#### DIS

In [9]:
dis_pfoa = mf6.ModflowGwtdis(
    gwt_pfoa,
    pname='dis',
    filename='peterson_pfoa.dis',
    length_units='METERS',
    nlay=dis.nlay.data,
    nrow=dis.nrow.data,
    ncol=dis.ncol.data,
    delr=20,
    delc=20,
    top='data-files/dis-top-elev.dat',
    botm='data-files/dis-bedrock-elev.dat',
    idomain='data-files/dis-idomain.dat',
    xorigin=dis.xorigin.data,
    yorigin=dis.yorigin.data,
    angrot=dis.angrot.data
)

dis_pfhxs = mf6.ModflowGwtdis(
    gwt_pfhxs,
    pname='dis',
    filename='peterson_pfhxs.dis',
    length_units='METERS',
    nlay=dis.nlay.data,
    nrow=dis.nrow.data,
    ncol=dis.ncol.data,
    delr=20,
    delc=20,
    top='data-files/dis-top-elev.dat',
    botm='data-files/dis-bedrock-elev.dat',
    idomain='data-files/dis-idomain.dat',
    xorigin=dis.xorigin.data,
    yorigin=dis.yorigin.data,
    angrot=dis.angrot.data
)

#### IC

In [10]:
ic_pfoa = flopy.mf6.ModflowGwtic(
    gwt_pfoa,
    strt=0,
    pname='ic',
)

ic_pfhxs = flopy.mf6.ModflowGwtic(
    gwt_pfhxs,
    strt=0,
    pname='ic',
)

#### ADV 

In [11]:
adv_pfoa = flopy.mf6.ModflowGwtadv(
    gwt_pfoa,
    scheme=adv.scheme.data,
    pname='adv',
)

adv_pfhxs = flopy.mf6.ModflowGwtadv(
    gwt_pfhxs,
    scheme=adv.scheme.data,
    pname='adv',
)

#### DSP

In [12]:
alh =  {'filename':'data-files/tran-dispersion.dat', 'factor': 1.0}
ath1 = {'filename':'data-files/tran-dispersion.dat', 'factor': 0.1}
atv =  {'filename':'data-files/tran-dispersion.dat', 'factor':0.01}

dsp_pfoa = flopy.mf6.ModflowGwtdsp(
    gwt_pfoa,
    alh=alh,
    ath1=ath1,
    atv=atv,
    pname='dsp',
)

dsp = flopy.mf6.ModflowGwtdsp(
    gwt_pfhxs,
    alh=alh,
    ath1=ath1,
    atv=atv,
    pname='dsp',
)

#### IST

In [13]:
ist_pfoa = flopy.mf6.ModflowGwtist(
    gwt_pfoa,
    pname='ist',
    sorption='LINEAR',
    porosity='data-files/tran-immobile-porosity.dat',
    volfrac=0.1,
    zetaim=0.001,
    bulk_density='data-files/tran-bulk-density.dat',
    distcoef=0.002754
)

ist_pfhxs = flopy.mf6.ModflowGwtist(
    gwt_pfhxs,
    pname='ist',
    sorption='LINEAR',
    porosity='data-files/tran-immobile-porosity.dat',
    volfrac=0.1,
    zetaim=0.001,
    bulk_density='data-files/tran-bulk-density.dat',
    distcoef=0.002754
)

### Create Non-Unique packages

#### MST

In [14]:
mst_pfoa = flopy.mf6.ModflowGwtmst(
    gwt_pfoa,
    sorption='LINEAR',
    porosity='data-files/tran-porosity.dat',
    bulk_density='data-files/tran-bulk-density.dat',
    distcoef='data-files/tran-Kd-pfoa.dat'
)

mst_pfhxs = flopy.mf6.ModflowGwtmst(
    gwt_pfhxs,
    sorption='LINEAR',
    porosity='data-files/tran-porosity.dat',
    bulk_density='data-files/tran-bulk-density.dat',
    distcoef='data-files/tran-Kd-pfhxs.dat'
)

#### SSM

In [15]:
ssm_pfoa = flopy.mf6.ModflowGwtssm(
    gwt_pfoa, 
    pname='ssm',
    sources=['rch', 'AUX', 'CONC_PFOA']
)

ssm_pfhxs = flopy.mf6.ModflowGwtssm(
    gwt_pfhxs, 
    pname='ssm',
    sources=['rch', 'AUX', 'CONC_PFHxS']
)

#### OC

In [16]:
oc_pfoa = flopy.mf6.ModflowGwtoc(
    gwt_pfoa,
    pname='oc_pfoa',
    concentration_filerecord='../output-files/PFOA_conc.ucn',
    saverecord= ['CONCENTRATION', 'ALL'],
    printrecord= ['CONCENTRATION', 'ALL']
)

oc_pfhxs = flopy.mf6.ModflowGwtoc(
    gwt_pfhxs,
    pname='oc_pfhxs',
    concentration_filerecord='../output-files/PFHxS_conc.ucn',
    saverecord= ['CONCENTRATION', 'ALL'],
    printrecord= ['CONCENTRATION', 'ALL']
)

### Exchange Files

In [17]:
exg_pfoa = flopy.mf6.ModflowGwfgwt(
    sim, exgmnamea='peterson_flow', exgmnameb=pfoa_name, filename='peterson_pfoa.gwfgwt'
)

exg_pfhxs = flopy.mf6.ModflowGwfgwt(
    sim, exgmnamea='peterson_flow', exgmnameb=pfhxs_name, filename='peterson_pfhxs.gwfgwt'
)

In [18]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package peterson_flow...
  writing solution package peterson_pfos...
  writing solution package ims_pfoa...
  writing solution package ims_pfhxs...
  writing package gwf-gwt_exg_0...
  writing package peterson_pfoa.gwfgwt...
  writing package peterson_pfhxs.gwfgwt...
  writing model peterson_flow...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package riv...
    writing package obs...
    writing package chd...
    writing package sto...
    writing package rcha_tas...
    writing package rcha...
    writing package oc...
    writing package rch_1_ts...
    writing package rch...
  writing model peterson_pfos...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package adv...
    writing package dsp...
    writing package mst...
    writing package ist..

In [20]:
## Uncomment this cell to run the simulation with the flow and PFOS transport model

success, buff = sim.run_simulation(silent=True, report=True, use_async=True)
assert success

AssertionError: 

In [21]:
buff

['(elapsed:0.041399)-->modflow 6',
 '(elapsed:7.400000000000462e-05)-->u.s. geological survey modular hydrologic model',
 '(elapsed:0.00012199999999999711)-->version 6.5.0 05/23/2024',
 '(elapsed:1.1999999999998123e-05)-->modflow 6 compiled may 23 2024 18:06:57 with intel(r) fortran intel(r) 64',
 '(elapsed:5.0000000000050004e-06)-->compiler classic for applications running on intel(r) 64, version 2021.7.0',
 '(elapsed:9.999999999996123e-06)-->build 20220726_000000',
 '(elapsed:7.000000000000062e-06)-->this software has been approved for release by the u.s. geological',
 '(elapsed:4.000000000004e-06)-->survey (usgs). although the software has been subjected to rigorous',
 '(elapsed:4.9999999999980616e-06)-->review, the usgs reserves the right to update the software as needed',
 '(elapsed:3.9999999999970615e-06)-->pursuant to further analysis and review. no warranty, expressed or',
 '(elapsed:4.000000000004e-06)-->implied, is made by the usgs or the u.s. government as to the',
 '(elapse